In [88]:
import jieba
import json
import random

In [48]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

In [49]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [50]:
# Find top10
res = es.search(index='facebook', doc_type='user', body= {'query': {'match': {'gender': 'female'}}})
print("%d documents found" % res['hits']['total'])

58 documents found


In [51]:
# Take all result
users = list(elasticsearch.helpers.scan(es, index='facebook', doc_type='user'))
total_user = [(user['_id'], user['_source'].get('gender')) for user in users]
print(len(total_user))

143


In [52]:
user_dict = {'male': [], 'female': []}
for user in total_user:
    if user[1] == 'male':
        user_dict['male'].append(user[0])
    else:
        user_dict['female'].append(user[0])

In [53]:
print('male: ' + str(len(user_dict['male'])) + '\nfemale: ' + str(len(user_dict['female'])))

male: 85
female: 58


In [62]:
# Mapping user posts
def get_posts(users):
    posts_dic = {}
    for user in users:
        # print(user)
        posts = list(elasticsearch.helpers.scan(es, query={'query': {'match': {'user_id': user}}}, index='facebook', doc_type='post'))
        message = [post['_source'].get('message') for post in posts if post['_source'].get('message') != '']
        #print(message)
        posts_dic[user] = message
    return posts_dic

In [63]:
male_posts_dict = get_posts(user_dict['male'])
female_posts_dict = get_posts(user_dict['female'])

In [82]:
# Check if content is Chinese or not
def is_Chinese(uchar):
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':  
        return True  
    else:  
        return False

In [95]:
for key, messages in male_posts_dict.items():
    # print(len(messages))
    random_num = random.sample(range(0, len(messages)), 5)
    res = [is_Chinese(messages[i]) for i in random_num]
    if (res.count(True)) < (res.count(False)):
        print(key)

1718957978189238
1695358370488750
1744561298891803
1759735150710515
1624663430897732
10213401737335928
10203448986903044
1532260126834371
1372981062791429
10213721570903289
1594133623932600
10154597524691479
1637654792912553
1746678138681672
10155404471078548
